<a href="https://colab.research.google.com/github/pinilDissanayaka/Psychology-RAG-Fusion/blob/main/Notebook_RAG_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain_groq pinecone langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [14]:
import os
from langchain_groq import ChatGroq
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

In [3]:
def getLlm():
  os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
  llm=ChatGroq(model_name='llama-3.1-70b-versatile',
              temperature=0.6)
  return llm

In [4]:
llm=getLlm()
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_time': 0.04, 'prompt_time': 0.005924067, 'queue_time': None, 'total_time': 0.045924067}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-84252997-b8a7-4156-bf0d-09d2e2a792d3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21})

In [5]:
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
pinecone=Pinecone()

In [6]:
pineconeIndexNames=pinecone.list_indexes().names()

print('Available Indexes  :')
print(f'\t {pineconeIndexNames}')

Available Indexes  :
	 ['american-history', 'wiki', 'pdf-rag', 'time-table']


In [8]:
INDEX_NAME='rag-fusion'
DIMENSIONS=512

if not INDEX_NAME in pineconeIndexNames:
  pinecone.create_index(
          name=INDEX_NAME,
          dimension=DIMENSIONS,
          metric="euclidean",
          spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
          )
)
else:
  print("Index already exists")

In [10]:
pinecone.describe_index(INDEX_NAME)

{'dimension': 512,
 'host': 'rag-fusion-4myrn7y.svc.aped-4627-b74a.pinecone.io',
 'metric': 'euclidean',
 'name': 'rag-fusion',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [12]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

ragFusionPrompt = ChatPromptTemplate.from_template(template)

In [13]:
ragFusionPrompt

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are a helpful assistant that generates multiple search queries based on a single input query. \n\nGenerate multiple search queries related to: {question} \n\nOutput (4 queries):'))])